In [ ]:
#| default_exp learner

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export 
import math, torch, matplotlib.pyplot as plt
import fastcore.all as fc
from collections.abc import Mapping
from operator import attrgetter
from functools import partial
from copy import copy

from torch import optim
import torch.nn.functional as F

from miniai.conv import *

from fastprogress import progress_bar, master_bar

In [ ]:
import matplotlib as mpl
import torchvision.transforms.functional as TF
from contextlib import contextmanager
from torch import nn, tensor
from datasets import load_dataset, load_dataset_builder
from miniai.datasets import *
import logging
from fastcore.test import test_close

In [ ]:
torch.set_printoptions(precision=2, linewidth=160, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'

In [ ]:
logging.disable(logging.WARNING)

In [ ]:
plt.style.use('dark_background')

## Learner

In [ ]:
x, y = 'image', 'label'
name = 'fashion_mnist'
dsd = load_dataset(name)

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset fashion_mnist downloaded and prepared to /root/.cache/huggingface/datasets/fashion_mnist/fashion_mnist/1.0.0/8d6c32399aa01613d96e2cbc9b13638f359ef62bb33612b077b4c247f6ef99c1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Grab a single example from Dataset and check its shape. We will be working with flattened images, so we convert PIL to tensor and flatten it out to get 784 (28x28) long tensor

In [ ]:
ex = TF.to_tensor(dsd['train'][0][x])
ex.shape

torch.Size([1, 28, 28])

In [ ]:
torch.flatten(TF.to_tensor(dsd['train'][0][x])).shape

torch.Size([784])

In [ ]:
@inplace
def transformi(b):
    # b is a dictionary of image and label
    # import ipdb; ipdb.set_trace()
    b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]

In [ ]:
bs = 1024
tds = dsd.with_transform(transformi)

In [ ]:
# remove number of workers during debugging
dls = DataLoaders.from_dd(tds, bs, num_workers=4)

dt = dls.train
xb, yb = next(iter(dt))
xb.shape, yb[:10]

(torch.Size([1024, 784]), tensor([5, 4, 9, 4, 3, 0, 6, 5, 7, 6]))

Let's remind ourselves how `DataLoaders.from_dd` actually works. 

```python
def from_dd(cls, dd, batch_size, as_tuple=True, **kwargs):
        f = collate_dict(dd['train'])
        return cls(*get_dls(*dd.values(), bs=batch_size, collate_fn=f, **kwargs))
```
Let's first deal with the first line of code and `f` function. `f` is an internal collation function that itself returns a function returning collated `x` and `y` values from a given batch. This is how it is done:
1. `collate_dict` takes feature names from `dd['train']`: `image` and `label` and puts them into an itemgetter
2.  this itemgetter is applied to the result of a `default_collate` on a given Dataset returning a tuple of `x` and `y` values

```python
def collate_dict(ds):
    get = itemgetter(*ds.features) # get x and y values
    def _f(b): return get(default_collate(b))
    return _f
```

The second line of code (returning) can be broken down into several parts: call `get_dls` and wrap its return value into `cls`. Let's tackle it one by one.

```python
def get_dls(train_ds, valid_ds, bs, **kwargs):
    return (DataLoader(train_ds, bs, shuffle=True, **kwargs), DataLoader(valid_ds, bs*2, **kwargs))
```

`get_dls` takes a train and valid datasets and turns them into the respective DataLoaders (PyTorch). Our created `collate_dict` function (assigned to `f`) is passed to the DataLoader `__init__` method via `**kwargs`

Wrapping the resulting tuple of DataLoaders inside `cls` simply allows us to get them by calling .train and .vaild on our Dataloaders class.

```python
def __init__(self, *dls): self.train, self.valid = dls[:2]
```

Now we should fully understand the initial code above. We create dataloaders (train and valid), then we select train dataloader and get one batche of size 1024 from it.

```python
dls = DataLoaders.from_dd(tds, bs)
dt = dls.train
xb, yb = next(iter(dt))
```
During the call to iterator our collation function (`f`) comes in play and merges the 1024 element list of dictionaries into a tuple of `x` and `y` tensors.

Now let's move on to creating our first Learner framework.

##  Learner

First we create a simplified learner that still has some nice structure to it.

In [ ]:
class Learner:
    def __init__(self, model, dls, loss_func, lr, opt_func=optim.SGD): fc.store_attr()
    
    def one_batch(self):
        self.xb, self.yb = to_device(self.batch)
        self.preds = self.model(self.xb)
        self.loss = self.loss_func(self.preds, self.yb)
        if self.model.training:
            self.loss.backward()
            self.opt.step()
            self.opt.zero_grad()
        with torch.no_grad(): self.calc_stats()
        
    def calc_stats(self):
        acc = (self.preds.argmax(dim=-1)==self.yb).float().sum()
        self.accs.append(acc)
        n = len(self.xb)
        self.losses.append(self.loss * n)
        self.ns.append(n)
          
    def one_epoch(self, train):
        self.model.training = train
        dl = self.dls.train if train else self.dls.valid
        for _, self.batch in enumerate(dl): self.one_batch()
        n = sum(self.ns)
        print(self.epoch, self.model.training, 
              sum(self.losses).item()/n, sum(self.accs).item()/n)
        
    def fit(self, n_epochs):
        self.accs, self.losses, self.ns = [], [], []
        self.model.to(def_device)
        self.opt = self.opt_func(self.model.parameters(), self.lr)
        self.n_epochs = n_epochs
        for self.epoch in range(n_epochs):
            self.one_epoch(True)
            with torch.no_grad(): self.one_epoch(False)

In [ ]:
m, nh = 28*28, 50
model = nn.Sequential(nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh, 10))

In [ ]:
learn = Learner(model, dls, F.cross_entropy, lr=0.2)
learn.fit(1)

0 True 1.17530390625 0.5987
0 False 1.1203112723214286 0.6135857142857143


## Basic Callbacks Learner 

Now let's improve greatly upon our previus iteration and add Callbacks to our Learner

In [ ]:
#|export
class CancelFitException(Exception): pass
class CancelBatchException(Exception): pass
class CancelEpochException(Exception): pass

Note that we set order variable outside `__init__` and so it belongs to the class and is shared by all instances.

In [ ]:
#|export
class Callback: order = 0

We use `getattr` to get a named attribute from an object; getattr(x, 'y') is equivalent to x.y.

In [ ]:
#|export
def run_cbs(cbs, method_nm, learn=None):
    for cb in sorted(cbs, key=attrgetter('order')):
        method = getattr(cb, method_nm, None)
        #?
        if method is not None: method(learn)

In [ ]:
class CompletionCB(Callback):
    def before_fit(self, learn): self.count = 0
    def after_batch(self, learn): self.count += 1
    def after_fit(self, learn): print(f'Completed {self.count} batches')

In [ ]:
cbs = [CompletionCB()]
run_cbs(cbs, 'before_fit')
run_cbs(cbs, 'after_batch')
run_cbs(cbs, 'after_fit')

Completed 1 batches


In [ ]:
class Learner:
    def __init__(self, model, dls, loss_func, lr, cbs, opt_func=optim.SGD): fc.store_attr()
    
    def one_batch(self):
        self.preds = self.model(self.batch[0])
        self.loss = self.loss_func(self.preds, self.batch[1])
        if self.model.training:
            self.loss.backward()
            self.opt.step()
            self.opt.zero_grad()
        
    def one_epoch(self, train: bool):
        self.model.train(train)
        self.dl = self.dls.train if train else self.dls.valid
        try:
            self.callback('before_epoch')
            for self.iter, self.batch in enumerate(self.dl):
                try:
                    self.callback('before_batch')
                    self.one_batch()
                    self.callback('after_batch')
                except CancelBatchException: pass
            # import ipdb; ipdb.set_trace()
            self.callback('after_epoch')
        except CancelEpochException: pass
            
    def fit(self, n_epochs):
        # import ipdb; ipdb.set_trace()
        self.n_epochs = n_epochs
        self.epochs = range(n_epochs)
        self.opt = self.opt_func(self.model.parameters(), self.lr)
        try:
            self.callback('before_fit')
            for self.epoch in self.epochs:
                self.one_epoch(True)
                self.one_epoch(False)
            self.callback('after_fit')
        except CancelFitException: pass
            
    def callback(self, method_nm): run_cbs(self.cbs, method_nm, self)

In [ ]:
m,nh = 28*28, 50
def get_model(): return nn.Sequential(nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh, 10))

In [ ]:
model = get_model()
learn = Learner(model, dls, F.cross_entropy, lr=0.2, cbs=[CompletionCB()])
learn.fit(1)

Completed 64 batches


In [ ]:
tds

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

```python
def get_dls(train_ds, valid_ds, bs, **kwargs):
    return (DataLoader(train_ds, bs, shuffle=True, **kwargs), DataLoader(valid_ds, bs*2, **kwargs))
```

In [ ]:
# 1024 * 2 because our dataloader for validation takes twice the batch size
math.ceil(60_000/1024) + math.ceil(10_000/(1024*2))

64

In [ ]:
#| export
class SingleBatchCB(Callback):
    order = 1
    def after_batch(self, learn): raise CancelFitException()

In [ ]:
learn = Learner(get_model(), dls, F.cross_entropy, lr=0.2, cbs=[SingleBatchCB(), CompletionCB()])
learn.fit(1)

## Metrics

We want to be able to use different metrics with our learner so we create a base class below.

In [ ]:
class Metric:
    def __init__(self): self.reset()
    
    def reset(self): self.vals, self.ns = [], []
    
    def add(self, inp, targ=None, n=1):
        self.last = self.calc(inp, targ)
        self.vals.append(self.last)
        self.ns.append(n)
    
    @property
    def value(self):
        ns = tensor(self.ns)
        # return weighted sum of metric
        return (tensor(self.vals)*ns).sum()/ns.sum()
        
    def calc(self, inps, targs): return inps

In [ ]:
class Accuracy(Metric):
    def calc(self, inps, targs): return (inps==targs).float().mean()

In [ ]:
acc = Accuracy()
acc.add(tensor([0,1,2,0,1,2]),tensor([0,1,1,2,1,0]))
acc.add(tensor([1,1,2,0,1]),tensor([0,1,1,2,1]))
acc.value

tensor(0.45)

Basic Metric can serve as a weighted average calculator because its `calc` method simply return input back.

In [ ]:
loss = Metric()
loss.add(0.6, n=32)
loss.add(0.9, n=2)
loss.value, round((0.6*32 + 0.9*2)/(32+2),2)

(tensor(0.62), 0.62)

## Some callbacks

Let's build upon the Metric class and add some metrics defined in PyTorch.

In [ ]:
#| export
from torcheval.metrics import MulticlassAccuracy, Mean

In [ ]:
metric = MulticlassAccuracy()
metric.update(tensor([0, 2, 1, 3]), tensor([0, 1, 2, 3]))
metric.compute()

tensor(0.50)

In [ ]:
metric.reset()
metric.compute()

tensor(nan)

Metrics are usually calculated using data that is on the `cpu`.

In [ ]:
#| export
def to_cpu(x):
    # import ipdb; ipdb.set_trace()
    """Recursively move data to cpu"""
    # mapping case
    if isinstance(x, Mapping): return {k: to_cpu(v) for k,v in x.items()}
    # list case
    if isinstance(x, list): return [to_cpu(o) for o in x]
    # tuple case (via list)
    if isinstance(x, tuple): return tuple(to_cpu(list(x)))
    # base case (recursive)
    res = x.detach().cpu()
    return res.float() if res.dtype == torch.float16 else res
                                          

In [ ]:
test = (tensor(1), tensor(2))

In [ ]:
to_cpu(test)

(tensor(1), tensor(2))

In [ ]:
#| export
class MetricsCB(Callback):
    def __init__(self, 
                 *ms, # list of metrics
                 **metrics, # dictionary of metrics
                ):
        # pool all unnamed metrics into a dictionary
        for o in ms: metrics[type(o).__name__] = o
        self.metrics = metrics
        self.all_metrics = copy(metrics)
        self.all_metrics['loss'] = self.loss = Mean()
        
    def _log(self, d): print(d)
    def before_fit(self, learn): learn.metrics = self
    def before_epoch(self, learn): [o.reset() for o in self.all_metrics.values()]
    
    # print metrics after each epoch
    def after_epoch(self, learn):
        # save log of all metrics
        log = {k: f'{v.compute():.3f}' for k,v in self.all_metrics.items()}
        log['epoch'] = learn.epoch
        log['train'] = 'train' if learn.model.training else 'eval'
        self._log(log)
    
    def after_batch(self,learn):
        # unpack all remaining values into *
        x,y,*_ = to_cpu(learn.batch)
        for m in self.metrics.values(): m.update(to_cpu(learn.preds), y)
        self.loss.update(to_cpu(learn.loss), weight=len(x))

In [ ]:
test = MetricsCB(1,'2',3.0,[4])
test.metrics

{'int': 1, 'str': '2', 'float': 3.0, 'list': [4]}

In [ ]:
def o(): return 1,2,3,4,5
x1, x2, *_ = o()

In [ ]:
#| export 
class DeviceCB(Callback):
    def __init__(self, device=def_device): fc.store_attr()
    def before_fit(self, learn):
        if hasattr(learn.model, 'to'): learn.model.to(self.device)
    def before_batch(self, learn): learn.batch = to_device(learn.batch, self.device)

In [ ]:
model = get_model()
metrics = MetricsCB(accuracy=MulticlassAccuracy())
learn = Learner(model, dls, F.cross_entropy, lr=0.2, cbs=[DeviceCB(), metrics])
learn.fit(1)

{'accuracy': '0.602', 'loss': '1.183', 'epoch': 0, 'train': 'train'}
{'accuracy': '0.700', 'loss': '0.847', 'epoch': 0, 'train': 'eval'}


## Flexible learner

Let's continue imporving upon our Learner. Below we discuss the key concepts that we will use.

loss1. __Contexmanager__

Context managers are objects that define the behavior to be performed when entering and exiting a context (e.g., acquiring and releasing resources). The contextmanager decorator simplifies the creation of context managers by eliminating the need to define a class with __enter__() and __exit__() methods.<br>

Inside contextmanager we use `yield` without any value. `yield` expression returns control to the whatever is using the generator. The generator pauses at this point, which means that the `@contextmanager` decorator knows that the code is done with the setup part. In other words, everything you want to do in the context manager `__enter__` phase has to take place before the yield.

Once the context exits (so the block under the with statement is done), the `@contextmanager` decorator is called for the `__exit__` part of the context manager protocol and will do one of two things:

* If there was no exception resume your generator. The generator unpauses at the yield line, and enter the cleanup phase
* If there was an exception, the decorator uses generator.throw() to raise that exception in the generator. It'll be as if the yield line caused that exception. Because we have a `finally` clause, it'll be executed before your generator exits because of the exception.

In our example the sequence is as follows:

1. with self.cb_ctx('epoch') creates the context manager and calls `__enter__` on that. 

2. The generator starts execution and tries calling `self.callback(f'before_{nm}')`

3. The `yield` expression pauses the generator, control goes back to the decorator. This takes whatever was yielded and returns that to the `with` statement, in case there is an `as target` part. Here `None` is yielded (there is only a plain `yield` expression).

4. Code below `with` statement:
```python
for self.iter, self.batch in enumerate(self.dl):
```
is run and completes.

5. The context manager __exit__ method is run, no exception is passed in.

6. The decorator resumes the generator, it continues where it left off.

7. The `finally` block is entered and `self.callback(f'cleanup_{nm}')` is executed

8. The generator exits, the decorator __exit__ method exits, all is done.


See https://stackoverflow.com/questions/35489844/what-does-yield-without-value-do-in-context-manager/35489897#35489897

In [ ]:
class Learner:
    def __init__(self, model, dls=(0,), loss_func=F.mse_loss, lr=0.1, cbs=None, opt_func=optim.SGD):
        # why do we need it? Does it affect fc.store_attr? It seems that is has no effect
        # cb = fc.L(cbs)
        fc.store_attr()
    
    @contextmanager
    def cb_ctx(self, nm):
        # import ipdb; ipdb.set_trace()
        try:
            self.callback(f'before_{nm}')
            yield
            self.callback(f'after_{nm}')
        except globals()[f'Cancel{nm.title()}Exception']: pass
        finally: self.callback(f'cleanup_{nm}')
            
    
    def one_epoch(self, train: bool):
        """Train for one epoch"""
        # set the model to the right mode and select relevant dataloader
        self.model.train(train)
        self.dl = self.dls.train if train else self.dls.valid
        # enter context for an epoch
        # import ipdb; ipdb.set_trace()
        with self.cb_ctx('epoch'):
            # loop through batches
            for self.iter, self.batch in enumerate(self.dl):
                # enter context for a batch
                with self.cb_ctx('batch'):
                    self.predict()
                    self.get_loss()
                    if self.training:
                        self.backward()
                        self.step()
                        self.zero_grad()

    
    def fit(self, n_epochs=1, train=True, valid=True, cbs=None, lr=None):
        # cast cbs to L
        cbs = fc.L(cbs)
        # if additional cbs are provided, pass it to the Learner for fitting and remove later
        for cb in cbs: self.cbs.append(cb)
        # try block
        try:
            # create number of epochs, their range and optimizer
            self.n_epochs = n_epochs
            self.epochs = range(n_epochs)
            self.opt = self.opt_func(self.model.parameters(), self.lr if lr is None else lr)
            # fit context
            # import ipdb; ipdb.set_trace()
            with self.cb_ctx('fit'):
                # train and validate for a given number of epochs
                for self.epoch in self.epochs:
                    if train: self.one_epoch(True)
                    # acts like a context manager with grad calculation disabled
                    if valid: torch.no_grad()(self.one_epoch)(False)
        finally:
            # remove cbs
             for cb in cbs: self.cbs.remove(cb)
      
    # __getattr__ method to call predict, get_loss, backward, step and zero_grad directly on Learner
    def __getattr__(self, name):
        if name in ("predict", "get_loss", "backward", "step", "zero_grad"): 
            # partial (instead of self.callback(name)) because we want to return a function, not a value (which is None)
            # run callback method that loops through all callbacks and calls those that have a specified name. For example `predict` from TrainCB
            return partial(self.callback, name)
        raise AttributeError(name)
       
    # run callback on a Learner
    def callback(self, method_nm): run_cbs(self.cbs, method_nm, self)
    
    # for ease of reference
    @property
    def training(self): return self.model.training                    

In [ ]:
#| export
class TrainCB(Callback):
    """Basic training callback"""
    # n_inp allows to train models with more than one input
    def __init__(self, n_inp=1): self.n_inp = n_inp
    def predict(self, learn): learn.preds = learn.model(*learn.batch[:self.n_inp])
    def get_loss(self, learn): learn.loss = learn.loss_func(learn.preds, *learn.batch[self.n_inp:])
    def backward(self, learn): learn.loss.backward()
    def step(self, learn): learn.opt.step()
    def zero_grad(self, learn): learn.opt.zero_grad()

In [ ]:
#| export
class ProgressCB(Callback):
    # decrease callback priority
    order = MetricsCB.order + 1
    def __init__(self, plot=False): self.plot = plot
    
    def before_fit(self, learn):
        # import ipdb; ipdb.set_trace()
        # create master_bar and set it to both mbar and learn.epochs
        learn.epochs = self.mbar = master_bar(learn.epochs)
        self.first = True
        # substitute _log method of learn's metrics (simple print) with progress bar
        if hasattr(learn, 'metrics'): learn.metrics._log = self._log
        self.losses = []
        self.val_losses = []
    
    def _log(self, d):
        if self.first:
            self.mbar.write(list(d), table=True)
            self.first = False
        self.mbar.write(list(d.values()), table=True)
        
    def before_epoch(self, learn): 
        # ??
        learn.dl = progress_bar(learn.dl, leave=False, parent=self.mbar)
        
    def after_batch(self, learn):
        learn.dl.comment = f'{learn.loss:.3f}'
        if self.plot and hasattr(learn, 'metrics') and learn.training:
            self.losses.append(learn.loss.item())
            if self.val_losses: 
                self.mbar.update_graph(
                    [[fc.L.range(self.losses), self.losses],
                     [fc.L.range(learn.epoch).map(lambda x: (x+1)*len(learn.dls.train)), 
                      self.val_losses]])
                   
    def after_epoch(self, learn):
        if not learn.training:
            if self.plot and hasattr(learn, 'metrics'):
                # import ipdb; ipdb.set_trace()
                # append to validation losses
                self.val_losses.append(learn.metrics.all_metrics['loss'].compute())
                self.mbar.update_graph(
                    # plot training losses
                    [[fc.L.range(self.losses), self.losses],
                     # plot validation losses, converting from epochs to batches
                     [fc.L.range(learn.epoch+1).map(lambda x: (x+1)*len(learn.dls.train)), 
                      self.val_losses]])

In [ ]:
model = get_model()

In [ ]:
metrics = MetricsCB(accuracy=MulticlassAccuracy())
cbs = [TrainCB(), DeviceCB(), metrics, ProgressCB(plot=True)]
learn = Learner(model, dls, F.cross_entropy, lr=0.2, cbs=cbs)
learn.fit(2)

accuracy,loss,epoch,train
0.600,1.179,0,train



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.9/bdb.py", line 334, in set_trace
    sys.settrace(self.trace_dispatch)



> /tmp/ipykernel_90/1864303131.py(41)after_epoch()
     40                 import ipdb; ipdb.set_trace()
---> 41                 self.val_losses.append(learn.metrics.all_metrics['loss'].compute())
     42                 self.mbar.update_graph(



ipdb>  s


--Call--
> /usr/local/lib/python3.9/dist-packages/torch/autograd/grad_mode.py(24)decorate_context()
     23 
---> 24         @functools.wraps(func)
     25         def decorate_context(*args, **kwargs):



...

--Return--
tensor(0.78, ...torch.float64)
> /usr/local/lib/python3.9/dist-packages/torch/autograd/grad_mode.py(27)decorate_context()
     26             with self.clone():
---> 27                 return func(*args, **kwargs)
     28         return cast(F, decorate_context)



ipdb>  n


> /tmp/ipykernel_90/1864303131.py(42)after_epoch()
     41                 self.val_losses.append(learn.metrics.all_metrics['loss'].compute())
---> 42                 self.mbar.update_graph(
     43                     [[fc.L.range(self.losses), self.losses],



ipdb>  q



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.9/bdb.py", line 359, in set_quit
    sys.settrace(None)

